## Just some basics

In [2]:
# Importing the Pytorch Library
import torch

Setting a seed helps in the reproducibility of the code.

In [3]:
torch.manual_seed(7)

How to create a tensor in Pytorch? It's pretty simple

In [4]:
torch.randn((2,4)) # Pass the dimensions of the tensor we want

tensor([[-0.1468,  0.7861,  0.9468, -1.1143],
        [ 1.6908, -0.8948, -0.3556,  1.2324]])

In [5]:
print(torch.randn((4,2,3))) 
print(torch.rand((2,3)))

tensor([[[-1.5827, -0.3246,  1.9264],
         [-0.3300,  0.1984,  0.7821]],

        [[ 1.0391, -0.7245, -0.1354],
         [ 0.7471,  0.6118,  1.8678]],

        [[ 2.5116, -1.2548,  0.8165],
         [-1.0654, -1.6370,  0.1577]],

        [[ 0.3957, -1.3677, -0.1007],
         [ 0.2370,  0.6327, -0.0917]]])
tensor([[0.2674, 0.4990, 0.7447],
        [0.7213, 0.4414, 0.5550]])


Get a tensor of same shape as another tensor?

In [6]:
a = torch.randn((2,3))
b = torch.randn_like(a) #_like implies that the new tensor will have same shape as the tensor passed to it
b.shape

torch.Size([2, 3])

**Note:** It's a great practice to keep an eye on the shape of the tensors when we are working with them, because most of the errors we face later (when doing complex NN architectures) are related to the proper shapes of tensors not being provided for a function down the road.

We can also easily pass tensors through functions. For example -

In [7]:
# Creating a sigmoid function
def activation(x):
    return 1/(1+torch.exp(-x))

In [9]:
a

tensor([[ 1.2202,  0.7588, -0.1857],
        [-0.8985,  0.6095,  0.2240]])

In [8]:
activation(a)

tensor([[0.7721, 0.6811, 0.4537],
        [0.2894, 0.6478, 0.5558]])

This is really helpful when we create complex neural networks, with lots of different activation functions being used.

### Tensor Multiplication

In [10]:
weights = torch.randn((1,5))
x = torch.randn_like(weights)
weights * x

tensor([[-0.7173,  0.0074, -0.1595, -0.7964, -0.3788]])

The simple multiplication gives us element wise multiplication. To get dot product, we can do a number of things, such as:

In [11]:
torch.sum(weights * x)

tensor(-2.0446)

In [12]:
(weights * x).sum()

tensor(-2.0446)

But when we will do deep learning, we will require to do huge matrix/tensor multiplications and it will be difficult to keep track of dimensions, so we will use torch.mm where mm stands for matrix multiplication instead. Moreover, these functions are more efficient in memory usage and have been implemented using modern libraries which accelerate the multiplication process as well.

In [13]:
torch.mm(weights, x)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at c:\a\w\1\s\tmp_conda_3.7_110509\conda\conda-bld\pytorch_1544094576194\work\aten\src\th\generic/THTensorMath.cpp:940

What's this error? As we can see that there is a size mismatch because for multiplying (a x b) and (c x d) matrix, both b and c should have same value (Look up matrix multiplication in general as to why if this is not clear). How to do that? Transpose!

In [14]:
torch.mm(weights, x.transpose(0,1)) # 0,1 are the dimensions being switched

tensor([[-2.0446]])

But there are better ways to tackle this problem, by using either of:

* `weights.reshape(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.
* `weights.resize_(a, b)` returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory. Here I should note that the underscore at the end of the method denotes that this method is performed **in-place**. Here is a great forum thread to [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)`.

I personally favour using .view() because it returns an error if we are trying to change the number of elements in a tensor when trying to reshape it and it keeps me from making such mistakes. But feel free to use whichever of them makes you comfortable.

In [16]:
torch.mm(weights, x.view(5,1))

tensor([[-2.0446]])

And next we can pass it through the activation function

In [17]:
activation(torch.mm(weights, x.view(5,1)))

tensor([[0.1146]])

Know what we did here? We created a simple neural network! (Albeit only the forward pass, but more on that later).
Next let's introduce a bias term as well.

Actually let's create a simple neural net with one hidden layer from scratch. 

In [19]:
torch.manual_seed(7)

x = torch.randn((1,3)) # We will have three inputs, x1, x2 and x3

input_size = x.shape[1] # Only take the number of columns (3 here)
hidden_size = 2 # Let's use 2 neurons in the hidden layer
output_size = 1 # We only want a single value as output

weights_for_hidden_layer = torch.randn(input_size, hidden_size) # The weights from input to hidden layer
weights_for_output_layer = torch.randn(hidden_size, output_size) # The weights from hidden to output layer

bias_for_hidden_layer = torch.randn(1,hidden_size) # we need to add bias to each neuron in the hidden layer
bias_for_output_layer = torch.randn(1,output_size) 

In [22]:
# Network creation

hidden_layer_output = activation(
    torch.mm(x, weights_for_hidden_layer) 
    + bias_for_hidden_layer)

output = activation(
    torch.mm(hidden_layer_output, weights_for_output_layer) 
    + bias_for_output_layer)

output

tensor([[0.3171]])

And this is how we create a simple neural network in Pytorch. Simple, right?

The trick is to take on one thing at a time and not trying to do everything at once. Do matrix multiplication first, see if its shape is right, then add the biases, then pass it through activation function. Repeat. Baby steps!